In [1]:
#!usr/bin/python
import girder_client
from utils import recurseGetResource, recurseGetItems
import re
from pprint import pprint
import json
import numpy as np

In [4]:
#FUNCTIONS

#write a function that renames the patient ID based on the specified format
#E0X to ADRC5X
#A10 equal to E10 which is ADRC60 (note that AXY is the same as EXY)
#OS0X to ADRC5X
#OS89 to ADRC39
#OS99 to ADRC49
#***Note that OS97 is ADRC56
#Also note that OSXY is the same 0SXY, this are just mistakes
#Also correct any _ to -
def correctPatientNames(currentName):
    currentName = currentName.replace('_','-')
    if currentName.find('ADRC') == -1:
        if currentName[0] == 'E' or currentName[0] == 'A':
            currentName = currentName.replace('A','ADRC')
            currentName = currentName.replace('E','ADRC')
        elif currentName.find('OS') == 0 or currentName.find('0S') == 0:
            currentName = currentName.replace('OS','ADRC')
            currentName = currentName.replace('0S','ADRC')
            if currentName[2:3] == '97':
                currentName[2:3] == '56'
            elif currentName[2] == '0':
                currentName[2] == '5'
            elif currentName[2] == '8':
                currentName[2] == '3'
            elif currentName[2] == '9':
                currentName[2] == '4'
    return currentName

#write a function that converts stain types to only one spelled variation of it (e.g. turns all H&E
#to HE to match others named that way)
def correctStainTypes(currentStainType):
    currentStainType = currentStainType.upper()
    currentStainType = re.sub('(?i)(ab|at8)','AB',currentStainType)
    currentStainType = re.sub('(?i)(he|h&e)','HE',currentStainType)
    currentStainType = re.sub('(?i)(pTPD|TDP)','TDP',currentStainType)
    currentStainType = re.sub('(?i)(s-m|sm)','SM',currentStainType)
    currentStainType = re.sub('(?i)(thiof|thoif)','THIO',currentStainType)
    currentStainType = re.sub('(?i)(thio|thoi)','THIO',currentStainType)
    return currentStainType

#write a function that takes in an dictionary of strings, containing patientID, blockID, staintype,
#slideFormat keys, and creates a filename template for that dictionary
def makefilename(m):
    return m['patientID']+'_'+m['blockID']+'_'+m['stainType']+'.'+m['slideFormat']

In [6]:
API_URL = "http://digitalslidearchive.emory.edu:8080/api/v1"
gc = girder_client.GirderClient(apiUrl=API_URL)
gc.authenticate(interactive=True) 

Login or email: admin
Password for admin: ········


In [7]:
folderToOrganize_id = '596e3b1492ca9a000b2af2b4'
## Folder is ADRC/.DG_ADRC_Slides

ADRC_SlideSet = recurseGetItems(gc,folderToOrganize_id) #this function was written by Mohammed and gives back
# each json object for each slide (in a array of json objects)

In [5]:
# save the ADRC_SlideSet in a numpy variable. This avoids running recurseGetItem, which takes a while.
# np.save('currentADRC_SlideSet', ADRC_SlideSet)

In [ ]:
#if you have the ADRC_SlideSet saved, load it to be used below
# ADRC_SlideSet = np.load('currentADRC_SlideSet.npy')

In [8]:
#compile the regex object to be used later
patientID_formats = 'A|E-|0S|ADRC|OS|E|A-'
stainType_formats = 'thiof|asyn|lfb|syn|ubiq|tau|biel|ab|AT8|thoi|thio|nf|sm|tdp|ptdp|ptpd|he|H&E|S-M'
# re_ADRC_SlideSet = re.compile('(?P<patientID>((A|E-|0S|ADRC|OS|E|A-)\d{1,3}-\d{1,3})|((A|E-|A-|0S|ADRC|OS|E)\d{1,3}_\d{1,3}))(_|-)(?P<blockID>.*)(?P<stainType>(?i)(thiof|asyn|lfb|syn|ubiq|tau|biel|ab|AT8|thoi|thio|nf|sm|tdp|ptdp|ptpd|he|H&E|S-M).*).(?P<slideFormat>(svs|ndpi))')
re_ADRC_SlideSet = re.compile('(?P<patientID>(('+patientID_formats+')\d{1,3}(-|_)\d{1,3}))(-|_)(?P<blockID1>.*)(?P<stainType>(?i)('+stainType_formats+'))(?P<blockID2>.*).(?P<slideFormat>(svs|ndpi))')
# re_ADRC_SlideSet = re.compile('(?P<patientID>(('+patientID_formats+')\d{1,3}(-|_)\d{1,3}))(-|_)(?P<blockID1>.*)(?P<stainType>(?i)('+stainType_formats+')).(?P<slideFormat>(svs|ndpi))')

In [9]:
#make a regex object that checks for the word control (case insensitive)
ignore_ADRC_SlideSet = re.compile('(?P<controlFlag>(\+|note|plus|neg|pos|minus|rename|control|cntrl|ctrl|contrl))', re.IGNORECASE)

In [14]:
successSlides = []
ignoreSlides = []
toBeSortedSlides = []
failedSlide = {'Attention':'This slide needs to be manually renamed and meta data added!',
              'patientID':'Blank', 'blockID':'Blank', 'stainType':'Blank', 'slideFormat':'Blank',
              'currentFileName': 'Blank', 'JCFileName': 'Blank'}
#slides w/ stain type asyn and syn are a problem since asyn is not fully caught cause syn takes precedence.
#to avoid always replace any asyn or a-syn found (case insensitive) w/ syn

for sld in ADRC_SlideSet:
    if ignore_ADRC_SlideSet.search(sld['name']): #find all control like words and ignore the slides
        ignoreSlides.append(sld)
    elif sld['name'].endswith(tuple(['.text','.xml','.txt'])): #unwanted file types
        ignoreSlides.append(sld)
    elif sld['name'].find('check') != -1: #this word gets caught as HE
        gc.addMetadataToItem(sld['_id'],failedSlide)
        toBeSortedSlides.append(sld)
    else:
        sld['name'] = re.sub('(?i)(asyn|a-syn)','syn',sld['name'])
        m = re_ADRC_SlideSet.search(sld['name'])
        #2 options: regex fit the current name or not
        if m:
            m_group = m.groupdict()
            if m_group['blockID1'].endswith(tuple(['-','_'])):
                m_group['blockID1'] = m_group['blockID1'][:-1]
#             if m_group['blockID2'].startswith(tuple(['-','_'])):
#                 m_group['blockID2'] = m_group['blockID2'][1:]
#             m_group['blockID'] = m_group['blockID1'] + m_group['blockID2']
            m_group['blockID'] = m_group['blockID1'].upper()
            m_group.pop('blockID1')
            m_group.pop('blockID2')
            m_group['patientID'] = correctPatientNames(m_group['patientID'])
            m_group['stainType'] = correctStainTypes(m_group['stainType'])
            m_group['JCFileName'] = makefilename(m_group)
            m_group['currentFileName'] = sld['name']
            m_group['Attention'] = None
            gc.addMetadataToItem(sld['_id'], m_group)
            sld['meta'] = m_group
            successSlides.append(sld)
        else:
            gc.addMetadataToItem(sld['_id'],failedSlide)
            toBeSortedSlides.append(sld)

In [18]:
print 'Number of slides that were successfully sorted: ', len(successSlides)
print 'Number of slides that need sorting: ',len(toBeSortedSlides)
print 'Number of slides ignored: ',len(ignoreSlides)

Number of slides that were successfully sorted:  7697
Number of slides that need sorting:  593
Number of slides ignored:  1985


In [16]:
#save the results of the current run as a json file
# with open('successSlides.json', 'w') as outfile:
#     json.dump(successSlides, outfile)
#     outfile.close()
# with open('toBeSortedSlides.json', 'w') as outfile2:
#     json.dump(toBeSortedSlides, outfile2)
#     outfile2.close()
# with open('ignoreSlides.json','w') as outfile3:
#     json.dump(ignoreSlides,outfile3)
#     outfile3.close()